In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import sklearn 

In [3]:
from sklearn.pipeline import Pipeline

In [17]:
df=pd.read_csv('final data set.csv',index_col=0)
df=df.drop(['Producer(s) rank',
'Director(s) rank',
'Writer(s) rank',
'Stars rank',
'Distributor rank','Worldwide Gross ($)','IMDB score'],axis=1)

,IMDB ID,Mojo ID,Year,Opening ($),Budget,Opening Theaters,In Realease (Days),Widest Release,Running time (min),IMDB score,...,country Swaziland,country Republic of North Macedonia,country Reunion,additive Producer(s) rank,additive Director(s) rank,additive Writer(s) rank,additive Stars rank,additive Distributor rank,Domestic Gross ($),Worldwide Gross ($)
0,tt0499549,876971521,2011.0,77025481.0,237000000.0,3452.0,379.0,3461.0,162.0,7.8,...,0.0,0.0,0.0,78817892.4,8757543.6,8757543.6,43787718.0,8757543.6,749766139.0,2.744337e+09
1,tt0435761,1383564801,2011.0,110307189.0,200000000.0,4028.0,203.0,4028.0,103.0,8.2,...,0.0,0.0,0.0,24951714.4,6237928.6,43665500.2,37427571.6,6237928.6,415004880.0,1.066970e+09
2,tt1014759,3393226241,2011.0,116101023.0,200000000.0,3728.0,302.0,3739.0,108.0,6.4,...,0.0,0.0,0.0,24671872.0,2467187.2,4934374.4,14803123.2,2467187.2,334191110.0,1.025467e+09
3,tt1228705,1515881985,2011.0,128122480.0,200000000.0,4380.0,239.0,4390.0,124.0,7.0,...,0.0,0.0,0.0,61036080.0,5086340.0,25431700.0,30518040.0,5086340.0,312433331.0,6.239333e+08
4,tt1325004,659654145,2011.0,64832191.0,68000000.0,4468.0,185.0,4468.0,124.0,5.0,...,0.0,0.0,0.0,10310985.0,1145665.0,2291330.0,6873990.0,1145665.0,300531751.0,6.984913e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7091,tt0104448,2835973633,2019.0,1273.0,NaN,1.0,306.0,1.0,57.0,6.2,...,0.0,0.0,0.0,4023.8,4023.8,4023.8,28166.6,4023.8,1273.0,1.273000e+03
7092,tt9434778,933790209,2019.0,546.0,NaN,1.0,385.0,2.0,79.0,6.3,...,0.0,0.0,0.0,1493.1,497.7,497.7,2488.5,497.7,4140.0,4.140000e+03
7093,tt3737840,621184513,2019.0,790.0,NaN,1.0,348.0,1.0,104.0,5.6,...,0.0,0.0,0.0,15775.2,1752.8,3505.6,8764.0,1752.8,790.0,7.900000e+02
7094,tt7543930,3389556225,2019.0,348.0,NaN,1.0,159.0,1.0,110.0,7.2,...,0.0,0.0,0.0,81496.8,11642.4,11642.4,58212.0,11642.4,1737.0,7.086596e+06


In [18]:
df

,IMDB ID,Mojo ID,Year,Opening ($),Budget,Opening Theaters,In Realease (Days),Widest Release,Running time (min),IMDB score,...,Writer(s) rank,Stars rank,Distributor rank,additive Producer(s) rank,additive Director(s) rank,additive Writer(s) rank,additive Stars rank,additive Distributor rank,Domestic Gross ($),Worldwide Gross ($)
0,tt0499549,876971521,2011.0,77025481.0,237000000.0,3452.0,379.0,3461.0,162.0,7.8,...,0.0,1.0,1.0,78817892.4,8757543.6,8757543.6,43787718.0,8757543.6,749766139.0,2.744337e+09
1,tt0435761,1383564801,2011.0,110307189.0,200000000.0,4028.0,203.0,4028.0,103.0,8.2,...,0.0,1.0,1.0,24951714.4,6237928.6,43665500.2,37427571.6,6237928.6,415004880.0,1.066970e+09
2,tt1014759,3393226241,2011.0,116101023.0,200000000.0,3728.0,302.0,3739.0,108.0,6.4,...,0.0,0.0,0.0,24671872.0,2467187.2,4934374.4,14803123.2,2467187.2,334191110.0,1.025467e+09
3,tt1228705,1515881985,2011.0,128122480.0,200000000.0,4380.0,239.0,4390.0,124.0,7.0,...,0.0,1.0,1.0,61036080.0,5086340.0,25431700.0,30518040.0,5086340.0,312433331.0,6.239333e+08
4,tt1325004,659654145,2011.0,64832191.0,68000000.0,4468.0,185.0,4468.0,124.0,5.0,...,0.0,0.0,0.0,10310985.0,1145665.0,2291330.0,6873990.0,1145665.0,300531751.0,6.984913e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7091,tt0104448,2835973633,2019.0,1273.0,NaN,1.0,306.0,1.0,57.0,6.2,...,0.0,0.0,0.0,4023.8,4023.8,4023.8,28166.6,4023.8,1273.0,1.273000e+03
7092,tt9434778,933790209,2019.0,546.0,NaN,1.0,385.0,2.0,79.0,6.3,...,0.0,0.0,0.0,1493.1,497.7,497.7,2488.5,497.7,4140.0,4.140000e+03
7093,tt3737840,621184513,2019.0,790.0,NaN,1.0,348.0,1.0,104.0,5.6,...,0.0,0.0,0.0,15775.2,1752.8,3505.6,8764.0,1752.8,790.0,7.900000e+02
7094,tt7543930,3389556225,2019.0,348.0,NaN,1.0,159.0,1.0,110.0,7.2,...,0.0,0.0,0.0,81496.8,11642.4,11642.4,58212.0,11642.4,1737.0,7.086596e+06


In [7]:
for item in df.columns:
    print(item )

IMDB ID
Mojo ID
Year
Opening ($)
Budget
Opening Theaters
In Realease (Days)
Widest Release
Running time (min)
IMDB score
IMDB votes
Metascore
genre Action
genre Adventure
genre Fantasy
genre Sci-Fi
genre Animation
genre Comedy
genre Family
genre Drama
genre Romance
genre Thriller
genre Mystery
genre Crime
genre Sport
genre Musical
genre Documentary
genre History
genre Music
genre Biography
genre Horror
genre War
genre Western
genre Short
genre News
genre Film-Noir
genre Reality-TV
genre Adult
MPAA PG-13
MPAA PG
MPAA R
MPAA G
MPAA Unrated
MPAA NC-17
MPAA Not Rated
MPAA M/PG
MPAA TV-PG
Day
Month
User
Critic
language English
language French
language Swedish
language Spanish
language Italian
language Hindi
language Hebrew
language German
language Arabic
language Korean
language None
language Mandarin
language Finnish
language Japanese
language Danish
language Vietnamese
language Portuguese
language Persian
language Greek
language Cantonese
language Serbian
language Norwegian
language Roman

1.0